## Indexado avanzado 


### Indexado con secuencias de índices

Consideremos un vector simple, y elijamos algunos de sus elementos

In [ ]:
x = np.linspace(0,3,7)
x

In [ ]:
# Standard slicing
v1=x[1::2]
v1

Esta es la manera simple de seleccionar elementos de un array, y como vimos lo que se obtiene es una vista del mismo array. **Numpy** permite además seleccionar partes de un array usando otro array de índices:

In [ ]:
# Array Slicing con índices ind
i1 = np.array([1,3,-1,0])   
v2 = x[i1]

In [ ]:
print(x)
print(x[i1])

In [ ]:
print(v1.base is x.base)
print(v2.base is x.base)

In [ ]:
x[[1,2,-1]]

Los índices negativos funcionan en exactamente la misma manera que en el caso simple. 

Es importante notar que cuando se usan arrays como índices, lo que se obtiene es un nuevo array (no una vista), y este nuevo array tiene las dimensiones (`shape`) del array de índices

In [ ]:
i2 = np.array([[1,0],[2,1]])
v3= x[i2]
print(x)
print(v3)
print('x  shape:', x.shape)
print('v3 shape:', v3.shape)

### Índices de arrays multidimensionales

In [ ]:
y = np.arange(12,0,-1).reshape(3,4)+0.5
y

In [ ]:
print(y[0])                     # Primera fila
print(y[2])                     # Última fila


In [ ]:
i = np.array([0,2])
print(y[i])       # Primera y última fila

Si usamos más de un array de índices para seleccionar elementos de un array multidimensional, cada array de índices se refiere a una dimensión diferente. Consideremos el array `y`

In [ ]:
print(y)

![](figuras/adv_index.png)

Si queremos elegir los elementos en los lugares `[0,1], [1,2], [0,3], [1,1]` (en ese orden) 
podemos crear dos array de índices con los valores correspondientes a cada dimensión

In [ ]:
i = np.array([0,1,0,1])
j = np.array([1,2,3,1])
print(y[i,j])

### Indexado con condiciones

Además de usar notación de *slices*, e índices también podemos seleccionar partes de arrays usando una matriz de condiciones. Primero creamos una matriz de coniciones `c`

In [ ]:
c = False*np.empty((3,4), dtype='bool')
print(c)

In [ ]:
# Es necesario dar el tipo de los elementos para que sean lógicos
False*np.empty((3,4))

In [ ]:
c[i,j]= True                    # Aplico la notación de índice avanzado
print(c)

In [ ]:
y

Como vemos, `c` es una matriz con la misma forma que `y`. Esto permite seleccionar los valores donde el array de condiciones es verdadero:

In [ ]:
yy = y[c]

In [ ]:
yy

In [ ]:
yy[0]=-2

In [ ]:
print(y)

Esta es una notación  potente. Por ejemplo, si en el array anterior queremos seleccionar todos los valores que sobrepasan cierto umbral (por ejemplo, los valores mayores a 7)

In [ ]:
print(y)
c1 = (y > 7)
print(c1)

El resultado de una comparación es un array donde cada elemento es un variable lógica (`True` o `False`). Podemos utilizarlo para seleccionar los valores que cumplen la condición dada. Por ejemplo

In [ ]:
y[c1]

De la misma manera, si queremos todos los valores entre 4 y 7 (incluidos), podemos hacer

In [ ]:
y[ (y >= 4) & (y <= 7) ]

Como mostramos en este ejemplo, no es necesario crear la matriz de condiciones previamente.

**Numpy** tiene funciones especiales para analizar datos de array que sirven para quedarse con los valores que cumplen ciertas condiciones. La función `nonzero` devuelve los índices donde el argumento no se anula:

In [ ]:
c1 = (y>=4) & (y <=7)
np.nonzero(c1)

Esta es la notación de avanzada de índices, y nos dice que los elementos cuya condición es diferente de cero (`True`) están en las posiciones: `[1,2], [1,3], [2,0]`. 

In [ ]:
indx, indy = np.nonzero(c1)
print('indx =', indx)
print('indy =', indy)

In [ ]:
for i,j in zip(indx, indy):
  print('y[{},{}]={}'.format(i,j,y[i,j]))

In [ ]:
print(np.nonzero(c1))
print(np.transpose(np.nonzero(c1)))
print(y[np.nonzero(c1)])

El resultado de `nonzero()` se puede utilizar directamente para elegir los elementos con la notación de índices avanzados, y su transpuesta es un array  donde cada elemento es un índice donde no se anula.

Existe la función `np.argwhere()` que es lo mismo que ``np.transpose(np.nonzero(a))``.

Otra función que sirve para elegir elementos basados en alguna condición es `np.compress(condition, a, axis=None, out=None)` que acepta un array unidimensional como condición

In [ ]:
c2 = np.ravel(c1)
print(c1)
print(c2)
print(y)
print(np.compress(c2,y))

In [ ]:
c3 = np.array(c2, dtype='int32')

In [ ]:
c3

In [ ]:
np.compress(c3 != 0,y)

La función `extract` es equivalente a convertir los dos vectores (condición y datos) a una dimensión (`ravel`) y luego aplicar `compress`

In [ ]:
np.extract(c1, y)

In [ ]:
print(y[c1])

### Función where

La función `where` permite operar condicionalmente sobre algunos elementos.  Por ejemplo, si queremos convolucionar el vector `y` con un escalón localizado en la región `[2,8]`:

In [ ]:
yy = np.where((y > 2) &  (y < 8) , y, 0)

In [ ]:
yy

Por ejemplo, para implementar la función de Heaviside

In [ ]:
import matplotlib.pyplot as plt

def H(x):
  return np.where(x < 0, 0, 1)
x = np.linspace(-1,1,11)
H(x)

In [ ]:
plt.plot(x,H(x), 'o')

## Extensión de las dimensiones (*Broadcasting*)

Vimos que en **Numpy** las operaciones (y comparaciones) se realizan "elemento a elemento". Sin embargo usamos expresiones del tipo `y > 4` donde comparamos un `ndarray` con un escalar. En este caso, lo que hace **Numpy** es extender automáticamente el escalar a un array de las mismas dimensiones que `y`

```python
  4 -> 4*np.ones(y.shape)
  ```

Hagamos esto explícitamente

In [ ]:
y

In [ ]:
y4 = 4*np.ones(y.shape)
np.all((y > y4) == (y > 4)) # np.all devuelve True si **TODOS** los elementos son iguales

De la misma manera, hay veces que podemos operar sobre arrays de distintas dimensiones

In [ ]:
y4

In [ ]:
y + y4

In [ ]:
y + 4

Como vemos eso es igual a `y + 4*np.ones(y.shape)`. En general, si Numpy puede transformar los arreglos para que todos tengan el mismo tamaño, lo hará en forma automática. 

Las reglas de la extensión automática son:

1. La extensión se realiza por dimensión. Dos dimensiones son compatibles si son iguales o una de ellas es 1.
2. Si los dos `arrays` difieren en el número de dimensiones, el que tiene menor dimensión se llena con `1` (unos) en el primer eje.

Veamos algunos ejemplos:


In [ ]:
x = np.arange(0,40,10)
xx = x.reshape(4,1)
y = np.arange(3)

In [ ]:
print(x.shape, xx.shape, y.shape)

In [ ]:
print(xx)

In [ ]:
print(y)

In [ ]:
print(xx+y)

Lo que está pasando es algo así como:

  * xx -> xxx
  * y ->  yyy
  * xx + y -> xxx + yyy

![](figuras/numpy_broadcasting.png)

donde `xxx`, `yyy` son versiones extendidas de los vectores originales:

In [ ]:
xxx = np.tile(xx, (1, y.size))
yyy = np.tile(y, (xx.size, 1))

In [ ]:
print(xxx)

In [ ]:
print(yyy)

In [ ]:
print(xxx + yyy)

## Unir (o concatenar) *arrays*

Si queremos unir dos *arrays* para formar un tercer *array* **Numpy** tiene una función llamada `concatenate`, que recibe una secuencia de arrays y devuelve su unión a lo largo de un eje.

### Apilamiento vertical

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8], [9,10]])
print('a=\n',a)
print('b=\n',b)

In [ ]:
# El eje 0 es el primero, y corresponde a apilamiento vertical
np.concatenate((a, b), axis=0)

In [ ]:
np.concatenate((a, b))          # axis=0 es el default

In [ ]:
np.vstack((a, b))    # Une siempre verticalmente (primer eje)

In [ ]:
np.stack((a,a))

Veamos cómo utilizar esto cuando tenemos más dimensiones. 

In [ ]:
c = np.array([[[1, 2], [3, 4]],[[-1,-2],[-3,-4]]])
d = np.array([[[5, 6], [7, 8]], [[9,10], [-5, -6]], [[-7, -8], [-9,-10]]])
print('c: shape={}\n'.format(c.shape),c)
print('\nd: shape={}\n'.format(d.shape),d)


Como tienen todas las dimensiones iguales, excepto la primera, podemos concatenarlos a lo largo del eje 0 (verticalmente)

In [ ]:
np.vstack((c,d))

In [ ]:
e=np.concatenate((c,d),axis=0)

In [ ]:
print(e.shape)
print(e)

### Apilamiento horizontal

Si tratamos de concatenar `a`y `b` a lo largo de otro eje vamos a recibir un error porque la forma de los `arrays` no es compatible.

In [ ]:
b.T

In [ ]:
print(a.shape, b.shape, b.T.shape)

In [ ]:
np.concatenate((a, b.T), axis=1)

In [ ]:
np.hstack((a,b.T))              # Como vstack pero horizontalmente

![](figuras/ilust_hstack.png) 

## Generación de números aleatorios

**Python** tiene un módulo para generar números al azar, sin embargo vamos a utilizar el módulo de **Numpy** llamado `random`. Este módulo tiene funciones para generar números al azar siguiendo varias distribuciones más comunes. Veamos que hay en el módulo

In [ ]:
dir(np.random)

### Distribución uniforme

Si elegimos números al azar con una distribución de probabilidad uniforme, la probabilidad de que el número elegido caiga en un intervalo dado es simplemente proporcional al tamaño del intervalo. 

In [ ]:
x= np.random.random((4,2))
y = np.random.random(8)
print(x)

In [ ]:
y

In [ ]:
help(np.random.random)

Como se infiere de este resultado, la función `random` (o `random_sample`) nos da una distribución de puntos aleatorios entre 0 y 1, uniformemente distribuidos.


In [ ]:
plt.plot(np.random.random(4000), '.')
plt.show()

In [ ]:
help(np.random.uniform)

### Distribución normal (Gaussiana)

Una distribución de probabilidad normal tiene la forma Gaussiana

$$p(x) = \frac{1}{\sqrt{ 2 \pi \sigma^2 }} e^{ - \frac{ (x - \mu)^2 } {2 \sigma^2} }.$$ En **Numpy** la función que nos da elementos con esa distribución de probabilidad es: 

`np.random.normal(loc=0.0, scale=1.0, size=None)`

donde:
 - `loc` es la posición del máximo (valor medio)
 - `scale` es el ancho de la distribución
 - `size` es el número de puntos a calcular (o forma)
 


In [ ]:
z = np.random.normal(size=4000)

In [ ]:
plt.plot( z, '.')
plt.show()

In [ ]:
np.random.normal(size=(3,5))

### Histogramas

Para visualizar los números generados y comparar su ocurrencia con la distribución de probabilidad 
vamos a generar histogramas usando *Numpy* y *Matplotlib*

In [ ]:
h,b = np.histogram(z, bins=20)

In [ ]:
b

In [ ]:
h

In [ ]:
b.size, h.size

La función retorna `b`: los límites de los intervalos en el eje x y `h` las alturas

In [ ]:
x = (b[1:] + b[:-1])/2

In [ ]:
plt.bar(x,h, align="center", width=0.4)
plt.plot(x,h, 'k', lw=4);
#plt.show()

**Matplotlib** tiene una función similar, que directamente realiza el gráfico

In [ ]:
h1, b1, p1 = plt.hist(z, bins=20)
#x1 = (b1[:-1] + b1[1:])/2
#plt.plot(x1, h1, '-k', lw=4)
plt.show()

In [ ]:
print(h1.size, b1.size)

Veamos otro ejemplo, agregando algún otro argumento opcional

In [ ]:
plt.hist(z, bins=20, density=True, orientation='horizontal', 
         alpha=0.8, histtype='stepfilled')
plt.show()

En este último ejemplo, cambiamos la orientación a `horizontal` y además normalizamos los resultados, de manera tal que la integral bajo (a la izquierda de, en este caso) la curva sea igual a 1.

### Distribución binomial

Cuando ocurre un evento que puede tener sólo dos resultados (verdadero, con probabilidad $p$, y falso con probabilidad $(1-p)$) y lo repetimos $N$ veces, la probabilidad de obtener el resultado con probabilidad $p$ es

$$
P(n) = \binom{N}{n}p^{n}(1-p)^{N-n},
$$

Para elegir números al azar con esta distribución de probabilidad **Numpy** tiene la función `binomial`,  cuyo primer argumento es $N$ y el segundo $p$. Por ejemplo si tiramos una moneda 100 veces, y queremos saber cuál es la probabilidad de obtener cara $n$ veces podemos usar:

In [ ]:
zb = np.random.binomial(100,0.5,size=30000)

In [ ]:
plt.hist(zb, bins=41, density=True, range=(30,70))
plt.xlabel('$n$ (veces "cara")')

In [ ]:
help(np.random.binomial)

Este gráfico ilustra la probabilidad de obtener $n$ veces un lado (cara) si tiramos 100 veces una moneda, como función de $n$.

-----

## Ejercicios 12 (b)

4. Vamos a estudiar la frecuencia de aparición de cada dígito en la serie de Fibonacci, generada siguiendo las reglas:
   $$a_{1} = a_{2} = 1, \quad a_{i} = a_{i-1} + a_{i-2}.$$
   Se pide:
   1. Crear una función que acepta como argumento un número entero $N$ y retorna una secuencia (lista, tupla, diccionario o *array*) con los elementos de la serie de Fibonacci.
   2. Crear una función que devuelva un histograma de ocurrencia de cada uno de los dígitos en el primer lugar del número. Por ejemplo para los primeros 8 valores ($N=8$): $1,1,2,3,5,8,13,21$ tendremos que el $1$ aparece 3 veces, el $2$ aparece $2$ veces, $3, 5, 8$ una vez. Normalizar los datos dividiendo por el número de valores $N$.
   3. Utilizando las dos funciones anteriores graficar el histograma para un número $N$ grande y comparar los resultados con la ley de Benford
  $$P(n) = \log_{10}\left(1+ \frac{1}{d} \right). $$


2. **PARA ENTREGAR:** Estimar el valor de π usando diferentes métodos basados en el método de Monte Carlo:

    1. Crear una función para calcular el valor de $\pi$ usando el "método de cociente de áreas". Para ello:

      * Generar puntos en el plano dentro del cuadrado de lado unidad cuyo lado inferior va de $x=0$ a $x=1$
      * Contar cuantos puntos caen dentro del (cuarto de) círculo unidad. Este número tiende a ser proporcional al área del círculo
      * La estimación de $\pi$ será igual a cuatro veces el cociente de números dentro del círculo dividido por el número total de puntos.

    2. Crear una función para calcular el valor de $\pi$ usando el "método del valor medio":
       Este método se basa en la idea de que el valor medio de una función se puede calcular de dos maneras diferentes.
       Por un lado es el promedio de los valores de la función si tomamos argumentos al azar en forma aleatoria con una distribución uniforme. Por otro lado, el valor medio verifica la siguiente fórmula integral:

    $$ \langle f \rangle = \frac{1}{b-a} \int_{a}^{b} f(x)\, dx $$

    Tomando la función particular $f(x)= \sqrt{1- x^{2}}$ entre $x=0$ y $x=1$, obtenemos:

    $$ \langle f \rangle = \int_{0}^{1} \sqrt{1- x^{2}}\, dx = \frac{\pi}{4} $$

    Entonces, tenemos que estimar el valor medio de la función $f$ y, mediante la relación entre las dos formas de calcular el valor medio obtener $\pi = 4 \langle f(x) \rangle$.

   Para obtener el valor medio de la función tomamos $X$ como una variable aleatoria entre 0 y 1 con distribución uniforme, y el valor promedio de $f(X)$ es justamente $\langle f \rangle$. Su función debe entonces

      * Generar puntos aleatoriamente en el intervalo $[0,1]$
      * Calcular el valor medio de $f(x)$ para los puntos aleatorios $x$.
      * El resultado va a ser igual al valor de la integral, y por lo tanto a $\pi/4$.

    3. Utilizar las funciones anteriores con diferentes valores para el número total de puntos $N$. En particular, hacerlo para 20 valores de $N$  equiespaciados logarítmicamente entre 100 y 10000. Para cada valor de $N$ calcular la estimación de $\pi$. Realizar un gráfico con el valor estimado como función del número $N$ con los dos métodos (dos curvas en un solo gráfico)

    4. Para $N=15000$ repetir el "experimento" muchas veces (al menos 1000) y realizar un histograma de los valores obtenidos para $\pi$ con cada método. Graficar el histograma y calcular la desviación standard. Superponer una función Gaussiana con el mismo ancho. El gráfico debe ser similar al siguiente (*el estilo de graficación no tiene que ser el mismo*)

    ![](figuras/ejercicio_09_1.png)
    
    5. El método de la aguja del bufón se puede utilizar para estimar el valor de $\pi$, y consiste en tirar agujas (o palitos, fósforos, etc) al azar sobre una superficie rayada

    ![](figuras/Streicholz-Pi-wiki.jpg)

    Por simplicidad vamos a considerar que la distancia entre rayas $t$ es mayor que la longitud de las agujas $\ell$

    ![](figuras/Buffon_needle_wiki.png)

    La probabilidad de que una aguja cruce una línea será:

    $$ P = \frac{2 \ell}{t\, \pi} $$

    por lo que podemos calcular el valor de $\pi$ si estimamos la probabilidad $P$. Realizar una función que estime $\pi$ utilizando este método y repetir las comparaciones de los dos puntos anteriores pero ahora utilizando este método y el de las áreas.
    
-----

> **NOTA:** Envíe el programa llamado **12_Suapellido.py** en un adjunto por correo electrónico, con asunto: **12_Suapellido**.
